## Chapter 1
# Digital Signals and Sampling

### Nyquist Sampling Theorem (Aliasing)

Nyquist's theorem states that the apparant frequency $f_a$ of continuous frequency $f$ sampled at frequency $f_s$ can be expressed as

$f_a = f - \lfloor \frac{f}{f_s} + \frac{1}{2} \rfloor f_s$

To see why, consider this animation, where the cycle on the left is sampled and stored in the cycle on the right.

The apparent and actual frequncies are charted below.  You can see that the apparent frequency is limited by $\frac{+}{-}\frac{f_s}{2}$.  That is, it's limited to $1/2$ of the sampling frequency.

_Notice that the times when apparent frequency is 0 are the times when the orange spinning line on the right appears to change direction from clockwise (negative) to counterclockwise (positive) motion._

There is a meta-sampling thing happening here, too! Since the blue "continuous" line is sampled at the animation's frame rate 40 FPS, when the blue line spins at $\frac{\text{FPS}}{2} = 20 Hz$ we see a distinct change in direction.  (This crossing point is shown in the second chart.)

In [7]:
import numpy as np

def apparent_frequency(f, f_s):
    return (f - np.floor(f / f_s + 0.5) * f_s)

In [1]:
import sys
sys.path.append('../')

from ipython_animation import create_animation
import matplotlib.pyplot as plt

%matplotlib inline

animation_length_seconds = 16
frames_per_second = 40
num_frames = animation_length_seconds * frames_per_second

continuous_coordinates = np.zeros((2, 2))
sampled_coordinates = np.zeros((2, 2))

continuous_coordinates[0,0] = -1
sampled_coordinates[0,0] = 1

fig = plt.figure(figsize=(10, 16))
plt.subplot(311)
continuous_line, = plt.plot(continuous_coordinates[0,:], continuous_coordinates[1,:], linewidth=4, label='Continuous signal')
sampled_line, = plt.plot(sampled_coordinates[0,:], sampled_coordinates[1,:], linewidth=4, label='Sampled signal')
plt.xlim(-2.1, 2.1)
plt.ylim(-1, 1)
plt.legend(loc='upper center')

f_s = 8
sample_frames = int(frames_per_second / f_s)
f_velocity = 1.5 / frames_per_second

plt.subplot(312)
plt.title('Apparent Frequency')

t_range = np.arange(num_frames, dtype='int')
f_history = np.ma.zeros(num_frames)
f_a_history = np.ma.zeros(num_frames)

f_line, = plt.plot(f_history, label='$f$')
f_a_line, = plt.plot(f_a_history, label='$f_a$')
plt.hlines(y=f_s / 2, xmin=0, xmax=num_frames, label='$\\frac{1}{2}$ sample rate ($f_s$)', color='r')
plt.hlines(y=frames_per_second / 2, xmin=0, xmax=num_frames, label='$\\frac{1}{2}$ animation FPS')
plt.legend(loc='upper left')

plt.ylim(-f_s / 2 - 0.1, f_velocity * num_frames + 0.1)

plt.subplot(313)
plt.title('Position')
pos_history = np.full(num_frames, np.nan)
sample_pos_history = np.full(num_frames, np.nan)
pos_line, = plt.plot(pos_history, label='actual')
sample_pos_line, = plt.plot(sample_pos_history, label='sample', marker='o', linewidth=3, markevery=sample_frames)
plt.xlim(0, num_frames)
plt.ylim(-1.1, 1.1)
plt.legend(loc='upper left')

theta_mutable = [0]

def animate(i):
    f = i * f_velocity
    f_history[i] = f
    f_a_history[i] = apparent_frequency(f, f_s)
    theta_mutable[0] += 2 * np.pi * f * (1 / frames_per_second)
    theta = theta_mutable[0]
    coordinates = [np.cos(theta), np.sin(theta)]
    continuous_coordinates[:,1] = continuous_coordinates[:,0] + coordinates
    pos_history[i] = coordinates[0]

    if i % sample_frames == 0:
        sampled_coordinates[:,1] = sampled_coordinates[:,0] + coordinates
        if i == 0:
            sample_pos_history[i] = pos_history[i]
        else:
            sample_pos_history[i - sample_frames:i + 1] = np.linspace(pos_history[i - sample_frames], pos_history[i], sample_frames + 1)
    continuous_line.set_data([continuous_coordinates[0,:], continuous_coordinates[1,:]])
    sampled_line.set_data([sampled_coordinates[0,:], sampled_coordinates[1,:]])
    f_line.set_ydata(np.ma.masked_where(t_range > i, f_history))
    f_a_line.set_ydata(np.ma.masked_where(t_range > i, f_a_history))
    
    pos_line.set_ydata(pos_history)
    sample_pos_line.set_ydata(sample_pos_history)
    
create_animation(fig, plt, animate, length_seconds=animation_length_seconds, frames_per_second=frames_per_second)

### Consequences of aliasing

Aliasing can cause distortion, since a low sample rate will not be able to represent high-frequency content.  For example, consider this synthesized violin tone with the following harmonics (from p17):

In [28]:
from NoteSequence import render_notes_ipython

violin_harmonics = [750, 1500, 2250, 3000, 3750, 4500, 5250, 6000, 6750, 7500, 8250, 9000, 9750]
violin_notes = [[(harmonic, 3, 1/(i + 1))] for i, harmonic in enumerate(violin_harmonics)]

render_notes_ipython(violin_notes)

This is the what would be the result of sampling at 10k samples-per-second:

In [29]:
sample_rate = 10_000
violin_harmonics_aliased = [np.abs(apparent_frequency(harmonic, sample_rate)) for harmonic in violin_harmonics]
print(violin_harmonics_aliased)
violin_notes_aliased = [[(harmonic, 3, 1/(i + 1))] for i, harmonic in enumerate(violin_harmonics_aliased)]

render_notes_ipython(violin_notes_aliased)

[750.0, 1500.0, 2250.0, 3000.0, 3750.0, 4500.0, 4750.0, 4000.0, 3250.0, 2500.0, 1750.0, 1000.0, 250.0]


These frequencies are no longer integer multiples of the fundamental, and thus no longer harmonics.